In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=2, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        #self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        
#         self.in_planes = 64
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
#                                stride=1, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
#         self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
#         self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
#         self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.layer4(out)
        #print(out.shape)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out



In [3]:
def project1_model():
    #return ResNet(BasicBlock, [1, 1, 1, 1])
    return ResNet(BasicBlock, [3, 3, 3])

In [4]:
# data transforms
dset_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])

In [5]:
# Use the image folder function to create datasets
dsets = {x: torchvision.datasets.CIFAR10(root='./ data', download=True, transform=dset_transform) 
         for x in ['train', 'val']}

# create data loader
dataloaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=128,
                                              shuffle=(x == "train"))
               for x in ['train', 'val']}

Files already downloaded and verified
Files already downloaded and verified


In [6]:
model=project1_model()
criterion  = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    model.cuda()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
from torchsummary import summary
summary(model)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            432
├─BatchNorm2d: 1-2                       32
├─Sequential: 1-3                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  2,304
|    |    └─BatchNorm2d: 3-2             32
|    |    └─Conv2d: 3-3                  2,304
|    |    └─BatchNorm2d: 3-4             32
|    |    └─Sequential: 3-5              288
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  2,304
|    |    └─BatchNorm2d: 3-7             32
|    |    └─Conv2d: 3-8                  2,304
|    |    └─BatchNorm2d: 3-9             32
|    |    └─Sequential: 3-10             --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 2,304
|    |    └─BatchNorm2d: 3-12            32
|    |    └─Conv2d: 3-13                 2,304
|    |    └─BatchNorm2d: 3-14            32
|    |    └─Sequential: 3-15             --
├─Seque

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            432
├─BatchNorm2d: 1-2                       32
├─Sequential: 1-3                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  2,304
|    |    └─BatchNorm2d: 3-2             32
|    |    └─Conv2d: 3-3                  2,304
|    |    └─BatchNorm2d: 3-4             32
|    |    └─Sequential: 3-5              288
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  2,304
|    |    └─BatchNorm2d: 3-7             32
|    |    └─Conv2d: 3-8                  2,304
|    |    └─BatchNorm2d: 3-9             32
|    |    └─Sequential: 3-10             --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 2,304
|    |    └─BatchNorm2d: 3-12            32
|    |    └─Conv2d: 3-13                 2,304
|    |    └─BatchNorm2d: 3-14            32
|    |    └─Sequential: 3-15             --
├─Seque

In [9]:

save_loss = {'train':[], 'val':[]}
save_acc = {'train':[], 'val':[]}

In [ ]:
for epoch in range(200):

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        current_loss = 0.0
        current_corrects = 0

        for batch_idx, (inputs, labels) in enumerate(dataloaders[phase], 1):
            optimizer.zero_grad()
            inputs=inputs.to(device)
            labels=labels.to(device)

            # Time to carry out the forward training poss
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # We want variables to hold the loss/acc statistics
            current_loss += loss.item() * inputs.size(0)
            current_corrects += torch.sum(preds == labels.data)
        # saving variable for plottin
        save_loss[phase] += [current_loss / len(dataloaders[phase].dataset)]
        save_acc[phase] += [current_corrects.float() / len(dataloaders[phase].dataset)]

        # pretty print
        print(f"Epoch:{epoch} -- Phase:{phase} -- Loss:{save_loss[phase][-1]:.2f} -- Acc:{save_acc[phase][-1]*100:.2f}")


Epoch:0 -- Phase:train -- Loss:1.56 -- Acc:41.75
Epoch:0 -- Phase:val -- Loss:1.35 -- Acc:51.10
Epoch:1 -- Phase:train -- Loss:1.08 -- Acc:61.16
Epoch:1 -- Phase:val -- Loss:1.22 -- Acc:58.24
Epoch:2 -- Phase:train -- Loss:0.87 -- Acc:69.06
Epoch:2 -- Phase:val -- Loss:0.94 -- Acc:66.54
Epoch:3 -- Phase:train -- Loss:0.74 -- Acc:73.99
Epoch:3 -- Phase:val -- Loss:1.45 -- Acc:57.19
Epoch:4 -- Phase:train -- Loss:0.65 -- Acc:77.07
Epoch:4 -- Phase:val -- Loss:0.71 -- Acc:74.80
Epoch:5 -- Phase:train -- Loss:0.58 -- Acc:79.76
Epoch:5 -- Phase:val -- Loss:0.61 -- Acc:78.89
Epoch:6 -- Phase:train -- Loss:0.53 -- Acc:81.61
Epoch:6 -- Phase:val -- Loss:0.63 -- Acc:77.95
Epoch:7 -- Phase:train -- Loss:0.48 -- Acc:83.40
Epoch:7 -- Phase:val -- Loss:0.50 -- Acc:82.72
Epoch:8 -- Phase:train -- Loss:0.44 -- Acc:84.62
Epoch:8 -- Phase:val -- Loss:0.49 -- Acc:83.09
Epoch:9 -- Phase:train -- Loss:0.40 -- Acc:86.09
Epoch:9 -- Phase:val -- Loss:0.66 -- Acc:77.05
Epoch:10 -- Phase:train -- Loss:0.37 -- 

In [ ]:
plt.plot(save_acc['train'])
plt.plot(save_acc['val'])
plt.legend(["train", "val"])
plt.title("Accuracy")
